In [ ]:
# ---------------------------
# 2.1 CRITICAL VALUES
# ---------------------------


def simulate_df_distribution(T, N, phi=1.0):
    """
    Simulate Dickey-Fuller test statistic distribution for log prices.
    
    Parameters:
    T (int): Length of each time series
    N (int): Number of Monte Carlo replications
    phi (float): AR(1) coefficient for log price process
    
    Returns:
    numpy array: Array of N test statistics
    """
    test_stats = np.zeros(N)

    for i in range(N):
        # 1. Simulate error terms εt ~ N(0, 1)
        errors = np.random.normal(0, 1, T)


        
        #-------------
        # # 2. Generate log price series pt = μ + φpt-1 + εt
        # log_prices = np.zeros(T)
        # log_prices[0] = errors[0]  # Initialize first value
        
        # # Generate log prices using AR(1) process
        # for t in range(1, T):
        #     log_prices[t] = phi * log_prices[t-1] + errors[t]

        #DO WE WANT TO USE THIS OR THE ONE BELOW?
        #_____________

        # 2. Generate log price series pt = μ + φpt-1 + εt
        log_prices = np.cumsum(errors) if phi == 1 else np.array([errors[0]] + [phi * log_prices[t-1] + errors[t] for t in range(1, T)])
        
        # 3. Estimate AR(1) model for log prices
        X = sm.add_constant(log_prices[:-1])  # Add constant and use lagged log prices
        y = log_prices[1:]  # Dependent variable is current log prices
        X= X[1:]
        y= y[1:]

        #------------------ HERE ALSO  ADDING X= X[1:] and y= y[1:] SHOULD ENSURE PROPER LAGGING 
        
        # Fit model
        model = sm.OLS(y, X).fit()
        
        # 4. Compute test statistic t(φ - 1)
        phi_hat = model.params[1]  # AR coefficient is second parameter (after constant)
        se_phi = model.bse[1]  # Standard error of AR coefficient
        test_stats[i] = (phi_hat - 1) / se_phi
    
    return test_stats  # Ensure function returns the results

# Set parameters for Monte Carlo simulation
N = 10000  # Number of Monte Carlo replications
T = 100  # Length of each time series (choose a reasonable value)

# Simulate for random walk in log prices (phi=1)
test_stats_rw = simulate_df_distribution(T, N, phi=1.0)

# Plot histogram
plt.figure(figsize=(10, 6))
sns.kdeplot(test_stats_rw, label="Empirical Distribution", color="blue")
plt.axvline(critical_values[0], color="red", linestyle="--", label="1% Critical Value")
plt.axvline(critical_values[1], color="orange", linestyle="--", label="5% Critical Value")
plt.axvline(critical_values[2], color="green", linestyle="--", label="10% Critical Value")
plt.title('Distribution of Dickey-Fuller Test Statistic\n(Random Walk in Log Prices)')
plt.xlabel('t(φ - 1)')
plt.ylabel('Density')
plt.legend()
plt.show()

# Compute critical values
critical_values = np.percentile(test_stats_rw, [1, 5, 10])
print("\nCritical Values:")
print(f"1%: {critical_values[0]:.3f}")
print(f"5%: {critical_values[1]:.3f}")
print(f"10%: {critical_values[2]:.3f}")

# Calculate theoretical critical values from t-distribution
theoretical_critical_values = t.ppf([0.01, 0.05, 0.10], df=T-1)
print("\nTheoretical Critical Values from t-distribution:")
print(f"1%: {theoretical_critical_values[0]:.3f}")
print(f"5%: {theoretical_critical_values[1]:.3f}")
print(f"10%: {theoretical_critical_values[2]:.3f}")

# Simulate for stationary AR(1) in log prices with phi=0.2
test_stats_ar = simulate_df_distribution(T, N, phi=0.2)

# Plot comparison of distributions
plt.figure(figsize=(9, 6))
plt.hist(test_stats_rw, bins=50, density=True, alpha=0.5, color='blue', label='Random Walk')
plt.hist(test_stats_ar, bins=50, density=True, alpha=0.5, color='red', label='AR(1) φ=0.2')
plt.title('Comparison of Test Statistic Distributions\n(Log Prices)')
plt.xlabel('t(φ - 1)')
plt.ylabel('Density')
plt.legend()
plt.show()

# Now simulate for T=500
T_500 = 500
test_stats_rw_500 = simulate_df_distribution(T_500, N, phi=1.0)

# Plot histogram for T=500
plt.figure(figsize=(9, 6))
plt.hist(test_stats_rw_500, bins=50, density=True, alpha=0.7, color='blue')
plt.title('Distribution of Dickey-Fuller Test Statistic\n(Random Walk in Log Prices, T=500)')
plt.xlabel('t(φ - 1)')
plt.ylabel('Density')
plt.show()

# Compute critical values for T=500
critical_values_500 = np.percentile(test_stats_rw_500, [1, 5, 10])
print("\nCritical Values (T=500):")
print(f"1%: {critical_values_500[0]:.3f}")
print(f"5%: {critical_values_500[1]:.3f}")
print(f"10%: {critical_values_500[2]:.3f}")